In [16]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI, AzureOpenAI
from IPython.display import Markdown, display
import openai
from PyPDF2 import PdfReader
import gradio as gr

load_dotenv()

/Users/anhv/projects/agentic-ai-engineering-course/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)
    
deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

In [6]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [7]:
print(linkedin)

   
Contact
Hanoi, Vietnam
brother.rain.1024@gmail.com
www.linkedin.com/in/vu-
anh-41254943  (LinkedIn)
undertheseanlp.com  (Personal)
Top Skills
Security
Technical Design
Natural Language Processing (NLP)
Languages
English  (Professional Working)
Vietnamese  (Native or Bilingual)
Certifications
Machine Learning
Introduction to Recommender
Systems
Improving Deep Neural Networks:
Hyperparameter tuning,
Regularization and Optimization
Neural Networks and Deep Learning
Structuring Machine Learning
Projects
Honors-Awards
Second Rank, VLSP Competition
AwardVu Anh
Lead AI Engineer - NLP
Vietnam
Summary
• Building Generative AI solutions
• Built Underthesea - the popular open-source Vietnamese NLP
toolkit (1500+ github stars)
My work centers on creating practical AI applications that bridge
cutting-edge research with real-world business value - from custom
LLM implementations to AI-powered automation systems that
enhance productivity and unlock new possibilities for enterprise
innovation.
Spe

In [9]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [10]:
name = "Vu Anh"

In [11]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [12]:
system_prompt

"You are acting as Vu Anh. You are answering questions on Vu Anh's website, particularly questions related to Vu Anh's career, background, skills and experience. Your responsibility is to represent Vu Anh for interactions on the website as faithfully as possible. You are given a summary of Vu Anh's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\n• Building Generative AI solutions\n\n• Built Underthesea - the popular open-source Vietnamese NLP toolkit (1500+ github stars)\n\nMy work centers on creating practical AI applications that bridge cutting-edge research with real-world business value - from custom LLM implementations to AI-powered automation systems that enhance productivity and unlock new possibilities for enterprise innovation.\n\nSpecialties:\n\n- AI: Generative AI, Chatbot Development, D

In [20]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [21]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [22]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [31]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [23]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [48]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = client.chat.completions.parse(model="gpt-4.1", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [42]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = client.chat.completions.parse(model="gpt-4.1-mini", messages=messages)
reply = response.choices[0].message.content

In [41]:
reply

'Thank you for your question. As of now, I do not hold any patents. However, I focus on building impactful AI solutions and open-source projects, such as Underthesea, which offers practical value to the community and businesses. If you have any questions about my work or expertise, feel free to ask!'

In [43]:
evaluate(reply, "do you hold a patent?", messages[:1])

'{"is_acceptable":true,"feedback":"The response is acceptable. It directly answers the user\'s question by stating that Vu Anh does not currently hold any patents. Additionally, it provides context about Vu Anh\'s focus and offers to engage further on topics related to patents in AI or NLP, which is professional and engaging. This meets the expectations for representing Vu Anh accurately and proactively on their website."}'

In [51]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [52]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [53]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response is written in what appears to be "Pig Latin," which is not appropriate or professional for a website representing Vu Anh, especially to an inquiry from a potential client or employer. The user asked about 'patent,' possibly looking for information on Vu Anh's patented work or experience with patents. The response should have been in clear, professional language, acknowledging the question, and either addressing experience related to patents or clarifying if there is no relevant experience. As is, the response is confusing and fails to engage the user properly.
Failed evaluation - retrying
The response is written in Pig Latin, which is not an appropriate or professional way to communicate on a website representing Vu Anh to potential clients, employers, or collaborators. The response should be clear, professional, and in the language the user expects (likely Vietnamese or English), especially regarding such di